In [1]:

import geopandas as gpd, pandas as pd
import contextily as ctx
from pyproj import CRS
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt
import ast
from collections import namedtuple

In [2]:
def create_named_tuple(filepath):
    """ generate a namedtuple from a txt file

    Parameters
        filepath: file path to .txt file
    Returns
        A namedtuple representing each path needed for system execution
    """
    file = open(filepath, "r")
    contents = file.read()
    dictionary = ast.literal_eval(contents)
    file.close()
    return namedtuple('_', dictionary.keys())(**dictionary)

In [3]:
def generate_custom_population(bins, labels, path_pop, path_region_names):
    """ generates age divided population

    Parameters
        bins: 1D array of bins in which to divide population
        labels: list of strings, names of age groups
        path_pop: path to population data
        path_region_names: path to region name data
    Returns
        dataframe with age divided population
    """
    total_pop = pd.read_csv(path_pop)
    age_divided = pd.DataFrame(total_pop.groupby(['region_id', pd.cut(total_pop["age"], bins=bins+[110], labels=labels, include_lowest=True)]).sum('population')['population'])
    age_divided.reset_index(inplace=True)
    age_divided = age_divided.pivot(index='region_id', columns=['age'])['population']
    region_names_id = pd.read_csv(path_region_names, delimiter=",").drop_duplicates()
    df = pd.merge(region_names_id, age_divided, on="region_id", how='right', sort=True)
    df['population'] = df.loc[:,df.columns[2:2+len(labels)]].sum(axis=1)
    return df

In [4]:

def generate_labels_from_bins(bins):
    """ generates labels for population dataframe

    Parameters
        bins: 1D array of bins to divide population
    Returns
        labels defining the population bins
    """
    labels = []
    for i in range(len(bins)-1):
        if i == 0:
            labels.append(str(bins[i])+"-"+str(bins[i+1]))
        else:
            labels.append(str(bins[i]+1)+"-"+str(bins[i+1]))
    labels.append(str(bins[-1]+1)+"+")
    return labels

In [5]:
paths = create_named_tuple('filepaths.txt')

In [6]:
config = create_named_tuple(paths.config)

In [7]:
age_labels = generate_labels_from_bins(config.age_bins)

In [8]:
 population = generate_custom_population(config.age_bins, age_labels, paths.age_divided_population, paths.municipalities_names)
 population['region_id'] = population['region_id'].astype('str')
 population = population[['region_id', 'population', 'region']]
 population

,region_id,population,region
0,301,696780,OSLO
1,1101,14785,EIGERSUND
2,1103,144115,STAVANGER
3,1106,37312,HAUGESUND
4,1108,80445,SANDNES
...,...,...,...
351,5440,928,BERLEVÅG
352,5441,2828,TANA
353,5442,880,NESSEBY
354,5443,2200,BÅTSFJORD


In [9]:
# read in geopandas
gdf = gpd.read_file('data/geospatial/municipalities_spatial_data.json')
gdf = gdf[['kommunenummer', 'geometry']]
df = pd.DataFrame(gdf)

# merge population and transform to geopandas 
gdf = gpd.GeoDataFrame(df.merge(population, right_on='region_id', left_on='kommunenummer',  suffixes=('', '_y')), geometry='geometry')
gdf = gdf.drop(columns=['kommunenummer'])
gdf = gdf.dropna() # very important in order to convert to crs
gdf = gdf.to_crs(3857)
gdf

,geometry,region_id,population,region
0,"POLYGON ((3226153.357 11202344.630, 3234057.61...",5405,5642,VADSØ
1,"POLYGON ((506175.483 8422940.082, 562484.473 8...",4625,5275,AUSTEVOLL
2,"POLYGON ((1572458.004 10430402.993, 1645715.33...",1848,2585,STEIGEN
3,"POLYGON ((1678017.887 10334098.090, 1682452.15...",1845,1909,SØRFOLD
4,"POLYGON ((3475684.021 11281761.823, 3511996.46...",5404,1958,VARDØ
...,...,...,...,...
357,"POLYGON ((980517.266 8366169.438, 986372.051 8...",3808,12989,NOTODDEN
358,"POLYGON ((989437.534 8287084.323, 999211.221 8...",3817,10459,MIDT-TELEMARK
359,"POLYGON ((1518588.435 9805613.431, 1522248.280...",1826,1266,HATTFJELLDAL
360,"POLYGON ((1103781.546 8307375.663, 1105728.799...",3802,25008,HOLMESTRAND


In [10]:
# random values
gdf['randNumCol'] = np.random.randint(1, 6, gdf.shape[0])
gdf

,geometry,region_id,population,region,randNumCol
0,"POLYGON ((3226153.357 11202344.630, 3234057.61...",5405,5642,VADSØ,5
1,"POLYGON ((506175.483 8422940.082, 562484.473 8...",4625,5275,AUSTEVOLL,4
2,"POLYGON ((1572458.004 10430402.993, 1645715.33...",1848,2585,STEIGEN,2
3,"POLYGON ((1678017.887 10334098.090, 1682452.15...",1845,1909,SØRFOLD,3
4,"POLYGON ((3475684.021 11281761.823, 3511996.46...",5404,1958,VARDØ,4
...,...,...,...,...,...
357,"POLYGON ((980517.266 8366169.438, 986372.051 8...",3808,12989,NOTODDEN,3
358,"POLYGON ((989437.534 8287084.323, 999211.221 8...",3817,10459,MIDT-TELEMARK,2
359,"POLYGON ((1518588.435 9805613.431, 1522248.280...",1826,1266,HATTFJELLDAL,5
360,"POLYGON ((1103781.546 8307375.663, 1105728.799...",3802,25008,HOLMESTRAND,4


In [11]:
# generate random data
res = np.random.rand(61, 8, 356)   #weeks, #compartments, #regions
res_accumulated_regions = res.sum(axis=2) #weeks, #compartments
num_weeks, num_compartments, num_regions = res.shape
compartment_labels = ['S', 'E1', 'E2', 'A', 'I', 'R', 'D', 'V']


In [12]:
# Matplotlib settings
params = {"axes.labelcolor":"slategrey"} 
plt.rcParams.update(params)
cmap = plt.cm.get_cmap("Reds")
blue = cmap(100)


In [13]:
# extract bounds from gdf 
west, south, east, north = gdf.total_bounds

In [14]:
res.shape

(61, 8, 356)

In [15]:
res_accumulated_regions.shape

(61, 8)

In [22]:
gdf['E1'].shape

(356,)

In [28]:
data = res[0,0,:]


In [29]:
gdf.plot(ax=ax, column=data, zorder=3)

<AxesSubplot:>

<Figure size 432x288 with 0 Axes>

In [18]:
# make the plots 
num_weeks = 50 # remove when done

for time_step in tqdm_notebook(range(num_weeks)):

    # Update values to plot in gdf
    ix_data = 4 # S, E1, E2, A, I, R, D, V
    data_to_plot = res[time_step, ix_data,:]

    # add axis for spatial plot
    fig, ax = plt.subplots(figsize=(14,14), dpi=72)
    gdf.plot(ax=ax, facecolor='none', edgecolor='gray', alpha=0.5, linewidth=0.5, zorder=3)
    gdf.plot(ax=ax, column='E1', zorder=3)
    
    # add background
    ctx.add_basemap(ax, zoom='auto', crs=3857, source=ctx.providers.Stamen.TonerLite, alpha=0.6, attribution="")
    ax.set_axis_off()
    ax.set_xlim(west, east)
    ax.set_ylim(south, north)
    ax.axis('off')
    plt.tight_layout()
    
    # axes for SEIR plot 
    inset_ax = fig.add_axes([0.6, 0.14, 0.37, 0.27])
    inset_ax.patch.set_alpha(0.5)

    # lines
    inset_ax.plot(res_accumulated_regions[:time_step, 0], label="S",  ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 1], label="E1", ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 2], label="E2", ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 3], label="A",  ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 4], label="I",  ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 5], label="R",  ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 6], label="D",  ls='-', lw=1.5, alpha=0.8)
    inset_ax.plot(res_accumulated_regions[:time_step, 7], label="V",  ls='-', lw=1.5, alpha=0.8)

    # fots on line
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 0], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 1], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 2], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 3], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 4], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 5], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 6], s=20, alpha=0.8)
    inset_ax.scatter((time_step-1), res_accumulated_regions[time_step, 7], s=20, alpha=0.8)

    # Shaded area and vertical dotted line between S and I curves in SEIR plot 
    #inset_ax.fill_between(np.arange(0, time_step), res_accumulated_regions[:time_step, 0].sum(axis=1), res_accumulated_regions[:time_step, 3].sum(axis=1), alpha=0.035, color='r')
    #inset_ax.plot([time_step, time_step], [0, max(res_accumulated_regions[(time_step-1), 0].sum(), res_accumulated_regions[(time_step-1), 3].sum())], ls='--', lw=0.7, alpha=0.8, color='r')
    
    # axes titles, label coordinates, values, font_sizes, grid, spines_colours, ticks_colurs, legend, title for SEIR plot
    inset_ax.set_ylabel('Population', size=14, alpha=1, rotation=90)
    inset_ax.set_xlabel('Weeks', size=14, alpha=1)
    inset_ax.yaxis.set_label_coords(-0.15, 0.55)
    inset_ax.tick_params(direction='in', size=10)
    inset_ax.set_xlim(-4, num_weeks)
    inset_ax.set_ylim(-24000, 5500000)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    inset_ax.grid(alpha=0.4)
    inset_ax.spines['right'].set_visible(False)
    inset_ax.spines['top'].set_visible(False)
    inset_ax.spines['left'].set_color('darkslategrey')
    inset_ax.spines['bottom'].set_color('darkslategrey')
    inset_ax.tick_params(axis='x', colors='darkslategrey')
    inset_ax.tick_params(axis='y', colors='darkslategrey')
    plt.legend(prop={'size':14, 'weight':'light'}, framealpha=0.5)
    plt.title("Covid-19 spreading in week: {}".format(time_step), fontsize=18, color= 'dimgray')

    plt.savefig("plots/flows_{}.jpg".format(time_step), dpi=fig.dpi)

<ipython-input-18-6bbc44ea3f6e>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for time_step in tqdm_notebook(range(num_weeks)):


  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-18-6bbc44ea3f6e>:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(14,14), dpi=72)


KeyboardInterrupt: 